# GBDT


## 参考资料

https://zhuanlan.zhihu.com/p/53980138
    
    
https://cloud.tencent.com/developer/article/1528767

https://hexinlin.top/2020/03/01/GBDT/
    
https://github.com/Freemanzxp/GBDT_Simple_Tutorial 配合 https://cloud.tencent.com/developer/article/1528336

https://blog.csdn.net/zpalyq110/article/details/79527653 （主要参考）

https://www.cnblogs.com/pinard/p/6143927.html 调参

https://cloud.tencent.com/developer/article/1587811

https://developer.aliyun.com/article/667318

## RF vs GBT

### 从模型框架的角度来看：

- 梯度提升树GBT 为boosting 模型。
- 随机森林RF  为bagging 模型。

### 从偏差分解的角度来看：

- 梯度提升树GBT 采用弱分类器（高偏差，低方差）。梯度提升树综合了这些弱分类器，在每一步的过程中降低了偏差，但是保持低方差。
- 随机森林RF 采用完全成长的子决策树（低偏差，高方差）。随机森林要求这些子树之间尽可能无关，从而综合之后能降低方差，但是保持低偏差。

### 如果在梯度提升树和随机森林之间二选一，几乎总是建议选择梯度提升树。

1. 随机森林的优点：天然的支持**并行**计算，因为每个**子树都是独立**的计算。

2. 梯度提升树的优点：

- 梯度提升树采用**更少的子树**来获得**更好的精度**。

  因为在每轮迭代中，梯度提升树会完全接受现有树（投票权为1）。而随机森林中每棵树都是同等重要的（无论它们表现的好坏），它们的投票权都是$1/N$ ，因此不是完全接受的。


- 梯度提升树也可以修改从而实现**并行化**。

- 梯度提升树有一个明确的数学模型。因此任何能写出梯度的任务，都可以应用梯度提升树（比如 ranking 任务）。而随机森林并没有一个明确的数学模型。

1. DT：回归树
使用CART回归树，可处理分类和回归问题。因为GBDT每次迭代拟合的都是梯度值，是连续值，所以是回归树。

关键：寻找最佳分割点

> 算法5.5（最小二乘回归树生成算法）  
输入：训练数据集$D$  
输出：回归树$f(x)$  
在训练数据集所在的输入空间中，递归地将每个区域划分为两个子区域并决定每个子区域上的输出值，构建二叉决策树；  
(1)选择最优切分变量$j$与切分点$s$，求解$$\min_{j,s} \left[ \min_{c_1} \sum_{x_i \in R_1(j,s)} (y_i - c_1)^2 + \min_{c_2} \sum_{x_i \in R_2(j,s)} (y_i - c_2)^2\right]$$遍历变量$j$，对固定的切分变量$j$扫描切分点$s$，选择使得上式达到最小值的对$(j,s)$  
(2)用选定的对$(j,s)$划分区域并决定相应的输出值：$$R_1(j,s)=\{x|x^{(j)}\leqslant s\}, R_2(j,s)=\{x|x^{(j)} > s\} \\ 
\hat{c_m} = \frac{1}{N_m} \sum_{x_i \in R_m(j,s)} y_i, x \in R_m, m=1,2 $$
(3)继续对两个子区域调用步骤(1),(2)，直至满足停止条件  
(4)将输入空间划分为$M$个区域$R_1,R_2,\cdots,R_M$，生成决策树：$$f(x)=\sum_{m=1}^M \hat{c_m} I(x \in R_m)$$

2. GB:拟合负梯度

通俗理解：假如有个人30岁，我们首先用20岁去拟合，发现损失有10岁，这时我们用6岁去拟合剩下的损失，发现差距还有4岁，第三轮我们用3岁拟合剩下的差距，差距就只有一岁了。如果我们的迭代轮数还没有完，可以继续迭代下面，每一轮迭代，拟合的岁数误差都会减小。最后将每次拟合的岁数加起来便是模型输出的结果。

**提升树算法**：

(1) 初始化$f_{0}(x)=0$

(2) 对$m=1,2,...,M$:(M为迭代轮数)

   (a) 计算残差：(N为样本数量)
   
$$r_{mi}=y_{i}-f_{m-1}(x_{i}),i=1,2,...,N$$
    
   (b)拟合残差 $r_{mi}$ 学习一个回归树，得到第$m$个决策树 $h_{m}(x)$
    
   (c)更新$f_{m}(x)=f_{m-1}(x)+h_{m}(x)$
   
(3) 得到回归提升树：

$$f_{M}(x)=\sum_{m=1}^{M}h_{m}(x)$$

第$m$个决策树的参数$\hat{\Theta _{m}}=argmin_{\Theta _{m}}\sum_{i=1}^{M}L(y_{i},f_{m-1}(x_i)+T(x_i;\Theta _{m}))$

$m-1$轮得到的分类器为$f_{m-1}(x)$，其损失函数为$L(y,f_{m-1}(x))$。$m$轮目标是找到一个弱学习器$h_{m}(x)$，最小化本轮损失$L(y,f_{m}(x))=L(y,f_{m-1}(x)+h_{m}(x))$.

如果采用平方误差损失即：

$$L(y,f_{m}(x))=L(y,f_{m-1}(x)+h_{m}(x))\\
=(y-f_{m-1}(x)-h_{m}(x))^{2}\\
=(r-h_{m}(x))^{2}
$$

其中$r=y-f_{m-1}(x)$

提升树中，当损失函数是平方损失函数和指数损失函数时，每一步优化都很简单。因为平方损失函数和指数损失函数的求导非常简单。

当损失函数是一般函数时，往往每一步优化不是很容易。针对这个问题，Freidman提出了梯度提升算法。

梯度提升树GBDT是利用最速下降法的近似方法。其关键是利用损失函数的负梯度在当前模型的值作为残差的近似值，从而拟合一个回归树。

—— http://www.huaxiaozhuan.com/%E7%BB%9F%E8%AE%A1%E5%AD%A6%E4%B9%A0/chapters/7_GBT.html

----

GBDT的关键：利用损失函数的**负梯度**作为提升树算法中的**残差的近似值**。

第$m$轮，第$i$个样本的loss的负梯度为：

$$-[\frac{\partial L(y,f(x_i))}{\partial f(x_i)}]_{f(x)=f_{m-1}(x)}$$

如果loss为平方误差损失$L(y,f(x_i))=\frac{1}{2}(y-f(x_i))^2$，则负梯度为（恰巧与残差一致，其他损失函数得到的负梯度是残差的近似）$$-[\frac{\partial L(y,f(x_i))}{\partial f(x_i)}]_{f(x)=f_{m-1}(x)}=y-f(x_i)$$

指数损失函数$L(y,f(x_i))=exp(-yf(x_i))$

分类任务的对数损失$$


## GBDT算法

(1) 初始化弱分类器：N为样本数量

$$f_{0}(x)=argmin_{c}\sum_{i=1}^{N}L(y_i,c)$$

(2) 对$m=1,2,...,M$:(M为迭代轮数)

&nbsp;&nbsp;&nbsp;&nbsp;(a) 对每个样本$i=1,2,...,N$，**计算残差即负梯度**(即模型优化方向)：
   
$$r_{mi}=-[\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)}]_{f(x)=f_{m-1}(x)}$$
    
其中N为样本数量，**$f(x)$**为当前大模型的输出，我们目前就是要对它进行改进。
    
&nbsp;&nbsp;&nbsp;&nbsp;(b)拟合残差 $r_{mi}$ 学习一个回归树，得到第$m$个决策树 $h_{m}(x)$

i. 将(a)步得到的残差$r_{mi}$作为样本新的真实值，并将数据$(x_i,r_{mi}),i=1,2,...,N$作为下棵树的训练数据，得到一颗新的回归树$h_{m}(x)$其对应的叶子节点区域为$R_{mj},j=1,2,...,J$。其中$J$为回归树$m$的叶子节点的个数。

ii. 对叶子区域$j=1,2,...,J$计算**最佳拟合值(优化的步长,$h_m(x)= \lambda _{m}* tree_m(x)$)**：

$$\gamma _{jm}=argmin_{\gamma }\sum_{x_i\in R_{jm}}L(y_i,f_{m-1}(x_i)+\gamma )$$

$$h_{m}(x) = \sum_{j=1}^{J}\gamma _{jm}I(x\in R_{jm})$$
    
其中$\gamma$ 表示优化的步长，$h_{m}(x)$为第$m$步的优化模型。
    
&nbsp;&nbsp;&nbsp;&nbsp;(c)更新强学习期：

$$f_{m}(x)=f_{m-1}(x)+h_{m}(x)$$
   
(3) 得到回归提升树：（回归树与分类树的差别在于损失函数）

$$f(x)=f_{M}(x)=f_0(x)+\sum_{m=1}^{M}h_{m}(x)\
=f_0(x)+\sum_{m=1}^{M}\sum_{j=1}^{J}\gamma _{jm}I(x\in R_{jm})$$


## GBDT另一种推导

先确定方向（负梯度），后确定走多远（学习率）。方向就是当前梯度下降的方向，走多远可以转为一个单变量的最值问题。

### GBDT分类

1. 确定下一轮迭代改进的方向：（gradient）

$$-\bigtriangledown err(y,G_M(x))$$

2. 用决策树（Decision Tree）模型$g_{M+1}(x)$拟合这个方向（负梯度）

3. 算出在负梯度这个方向上走多远：求解$\lambda _{M+1}$

4. 把$\lambda _{M+1}\cdot g_{M+1}(x)$添加到$G(x)=G_M(x)$中(即Boosting)

所以Adaboost就是GBDT的特例，当GBDT损失函数是指数损失时，就是Adaboost。

### GBDT回归

1. 首先定方向:$-\bigtriangledown err(y,G_M(x))=2(y-G_M(x))$

2. 让一个回归决策树$g_{M+1}(x)$拟合这个方向（负梯度）

3. $err(y,\hat{G}_M(x))=err(y,G_M(x)+\lambda _{M+1} \cdot g_{M+1}(x)) = (\lambda _{M+1} \cdot g_{M+1}(x) - (y-G_M(x)))^2$

4. 求解$\frac{\partial err(y,\hat{G}_M(x))}{\partial \lambda _{M+1}}=0$，得出$\lambda _{M+1}$

5. 把$\lambda _{M+1}\cdot g_{M+1}(x)$添加到原模型$G(x)$即$G_M(x)$中

## GBDT实例

### 1. 初始化弱分类器

$$f_{0}(x)=argmin_{c}\sum_{i=1}^{N}L(y_i,c)$$

损失函数为平方损失（凸函数），求导，令导数为零，得到$c$：

$$ \sum_{i=1}^{N}\frac{\partial L(y_i,c)}{\partial c} = \sum_{i=1}^{N}\frac{\partial (\frac{1}{2}(y_i-c)^2)}{\partial c}=\sum_{i=1}^{N}(c-y_i) $$

令上式为0，得到：

$$c=(\sum_{i=1}^{N}y_i)/N$$

所以初始化时，$c$取值为所有样本数据的label的均值：

$$f_0(x)=c$$

### 2.迭代轮数m=1,2,...,M

$M$为超参。

step1.计算负梯度，因为平方误差损失的负梯度就是残差（$y$减去上一轮得到的学习期$f_{m-1}$的的差值）：

$$r_{1i}=-[\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)}]_{f(x)=f_{0}(x)}$$

计算出每个样本$i=1,2,...,N$的残差并替代对应的label，训练若分类器$f_1(x)$

step2.寻找回归树最佳分割点。遍历每个特征的每个值。

如果有2个特征，每个特征有3个值，则需要计算至多6次平方损失。每个分割点将该特征上的特征值list一分为二，$ERR_{sum}=ERR_{left}+ERR_{right}$

选择$ERR_{sum}$最小的为划分点（如果有重复的就随便选一个）。

step3.

如果树深度(超参)没达到预设值，且有数据，继续重复step1和step2，分割下一层的节点。

如果达到树深度预设值，给每个叶子节点赋值一个参数$\gamma $拟合残差(假设m=1时结束分割)：

$$\gamma _{j1}=argmin_{\gamma }\sum_{x_i\in R_{j1}}L(y_i,f_{0}(x_i)+\gamma )$$

其中$j=1,2,...,J$表示叶子节点数。

将平方误差带入上式，对上式求导，令导数为零，得到$𝑐$即每个叶子节点的参数$\gamma $(也就是label的均值，但不是原始$y$，而是本轮要拟合的残差$y-f_0(x)$):

![example](https://img-blog.csdnimg.cn/20190421104545703.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3pwYWx5cTExMA==,size_16,color_FFFFFF,t_70)


此时，更新强分类器：

其中learningrate是超参数学习率。Shrinkage的思想，如果**每次都全部加上（学习率为1）很容易一步学到位导致过拟合**。

$$f_1(x)=f_0(x) + learningrate * \sum _{j=1}^{J}\gamma _{j1}I(x\in R_{j1}),J=4$$

最终的分类器为：

$$f(x)=f_{5}(x)=f_0(x)+\sum_{m=1}^{5}h_{m}(x)\
=f_0(x)+\sum_{m=1}^{5}\sum_{j=1}^{4}\gamma _{jm}I(x\in R_{jm})$$

In [2]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

'''
调参：
loss：损失函数。有deviance和exponential两种。deviance是采用对数似然，exponential是指数损失，后者相当于AdaBoost。
n_estimators:最大弱学习器个数，默认是100，调参时要注意过拟合或欠拟合，一般和learning_rate一起考虑。
learning_rate:步长，即每个弱学习器的权重缩减系数，默认为0.1，取值范围0-1，当取值为1时，相当于权重不缩减。较小的learning_rate相当于更多的迭代次数。
subsample:子采样，默认为1，取值范围(0,1]，当取值为1时，相当于没有采样。小于1时，即进行采样，按比例采样得到的样本去构建弱学习器。这样做可以防止过拟合，但是值不能太低，会造成高方差。
init：初始化弱学习器。不使用的话就是第一轮迭代构建的弱学习器.如果没有先验的话就可以不用管

由于GBDT使用CART回归决策树。以下参数用于调优弱学习器，主要都是为了防止过拟合
max_feature：树分裂时考虑的最大特征数，默认为None，也就是考虑所有特征。可以取值有：log2,auto,sqrt
max_depth：CART最大深度，默认为None
min_sample_split：划分节点时需要保留的样本数。当某节点的样本数小于某个值时，就当做叶子节点，不允许再分裂。默认是2
min_sample_leaf：叶子节点最少样本数。如果某个叶子节点数量少于某个值，会同它的兄弟节点一起被剪枝。默认是1
min_weight_fraction_leaf：叶子节点最小的样本权重和。如果小于某个值，会同它的兄弟节点一起被剪枝。一般用于权重变化的样本。默认是0
min_leaf_nodes：最大叶子节点数
'''

'\n调参：\nloss：损失函数。有deviance和exponential两种。deviance是采用对数似然，exponential是指数损失，后者相当于AdaBoost。\nn_estimators:最大弱学习器个数，默认是100，调参时要注意过拟合或欠拟合，一般和learning_rate一起考虑。\nlearning_rate:步长，即每个弱学习器的权重缩减系数，默认为0.1，取值范围0-1，当取值为1时，相当于权重不缩减。较小的learning_rate相当于更多的迭代次数。\nsubsample:子采样，默认为1，取值范围(0,1]，当取值为1时，相当于没有采样。小于1时，即进行采样，按比例采样得到的样本去构建弱学习器。这样做可以防止过拟合，但是值不能太低，会造成高方差。\ninit：初始化弱学习器。不使用的话就是第一轮迭代构建的弱学习器.如果没有先验的话就可以不用管\n\n由于GBDT使用CART回归决策树。以下参数用于调优弱学习器，主要都是为了防止过拟合\nmax_feature：树分裂时考虑的最大特征数，默认为None，也就是考虑所有特征。可以取值有：log2,auto,sqrt\nmax_depth：CART最大深度，默认为None\nmin_sample_split：划分节点时需要保留的样本数。当某节点的样本数小于某个值时，就当做叶子节点，不允许再分裂。默认是2\nmin_sample_leaf：叶子节点最少样本数。如果某个叶子节点数量少于某个值，会同它的兄弟节点一起被剪枝。默认是1\nmin_weight_fraction_leaf：叶子节点最小的样本权重和。如果小于某个值，会同它的兄弟节点一起被剪枝。一般用于权重变化的样本。默认是0\nmin_leaf_nodes：最大叶子节点数\n'

In [4]:
train_feat = np.array([[1, 5, 20],
                       [2, 7, 30],
                       [3, 21, 70],
                       [4, 30, 60],
                       ])
train_label = np.array([[0], [0], [1], [1]]).ravel()

test_feat = np.array([[5, 25, 65]])
test_label = np.array([[1]])
print(train_feat.shape, train_label.shape, test_feat.shape, test_label.shape)

(4, 3) (4,) (1, 3) (1, 1)


In [5]:
gbdt = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=5, subsample=1
                                  , min_samples_split=2, min_samples_leaf=1, max_depth=3
                                  , init=None, random_state=None, max_features=None
                                  , verbose=0, max_leaf_nodes=None, warm_start=False
                                  )
gbdt.fit(train_feat, train_label)

GradientBoostingClassifier(n_estimators=5, subsample=1)

In [6]:
pred = gbdt.predict(test_feat)
total_err = 0
for i in range(pred.shape[0]):
    print(pred[i], test_label[i])
    err = (pred[i] - test_label[i]) / test_label[i]
    total_err += err * err
print(total_err / pred.shape[0])

1 [1]
[0.]
